In [2]:
import json
import os
import re
import collections
import numpy as np
import pandas as pd
import tensorflow as tf
import LAC
import happybase
import joblib

In [5]:
df_list = []
#with open("/mnt1/zhaodachuan/data/predict_user_attribute20200911/raw_data/user_profile/position_data") as f:
with open(r"D:\zhaodachuan\data\predict_user_attribute20200911\raw_data\user_profile\trade_data") as f:
    for line in f:
        df_list.append(line.strip().split('\001'))
df = pd.DataFrame(df_list)
df

,0,1,2,3,4,5,6,7,8,9,10,11
0,10113,内容/营销/传播,378020,378036,377965,377980,378047,378021,377903,377873,378033,378025
1,10140,IT,381349,381108,381299,381252,380842,381326,379187,381311,381059,380919
2,101966,电商/仓储物流,381295,381108,381284,381313,381106,380842,381316,381279,381315,381310
3,103927,金融,365636,365655,365723,365476,365546,365554,365423,365698,365331,365761
4,10714,IT,376192,376167,377536,376189,376126,374016,373949,378198,374017,379878
...,...,...,...,...,...,...,...,...,...,...,...,...
6946,8,内容/营销/传播,379195,379107,379194,380776,378982,379529,379133,379489,379526,379004
6947,83135,金融,374584,354849,377736,371117,377233,381026,379256,378084,379411,381309
6948,84249,互联网,380858,379194,380797,380393,379607,380902,381078,379795,379977,378982
6949,89195,IT,358942,360290,360133,359550,368330,368145,363890,363980,367415,363881


In [16]:
df[1] = df[1].apply(lambda x : x.lower())

In [17]:
all_counter = collections.Counter([v for v in df[1]])

In [18]:
all_counter

Counter({'内容/营销/传播': 422,
         'it': 1286,
         '电商/仓储物流': 263,
         '金融': 947,
         '创业者': 410,
         '商业服务-o2o/服务/社区': 113,
         '互联网': 433,
         '文化/体育/娱乐业': 163,
         '投资人': 196,
         '教育/专业服务/培训': 358,
         '酒店/餐饮/旅游': 150,
         'tmt从业人员': 123,
         '仓储物流': 60,
         '政府/非盈利机构': 60,
         '商业服务': 208,
         '加工制造': 164,
         '汽车/出行': 98,
         '法律': 36,
         '其他': 384,
         '人工智能/前沿科技': 78,
         '媒体': 166,
         '智能硬件/硬件制造': 92,
         '能源相关': 60,
         '农业': 44,
         '房地产': 159,
         '制药医疗/生物/卫生保健': 137,
         '企业服务/云计算/大数据': 116,
         '学生': 219,
         '营销': 1,
         '贸易/零售批发': 2,
         '-1': 3})

In [19]:
def temp_function(x):
    #x = datetime.datetime.now().year - int(x.split("-")[0])
    y = 0
    if x in set(['it','互联网','tmt从业人员','人工智能/前沿科技','企业服务/云计算/大数据']):
        y = "IT/移动互联网"
    elif x in set(['教育/专业服务/培训','内容/营销/传播','营销','贸易/零售批发','酒店/餐饮/旅游']):
        y = "零售消费"
    elif x in set(['金融','投资人']):
        y = "金融"
    elif x in set(['汽车/出行',]):
        y = "汽车"
    elif x in set(['电商/仓储物流','仓储物流','商业服务','商业服务-o2o/服务/社区']):
        y = "电商"
    elif x in set(['加工制造']):
        y = "制造业"
    elif x in set(['智能硬件/硬件制造']):
        y = "IOT"
    elif x in set(['文化/体育/娱乐业']):
        y = "游戏"
    elif x in set(['房地产',]):
        y = "房地产"
    elif x in set(['制药医疗/生物/卫生保健',]):
        y = "医疗"
    elif x in set(['创业者','政府/非盈利机构','法律','其他','媒体','能源相关','农业','学生','-1']):
        y = "其他"
    else:
        raise
    return y

In [20]:
df[1] = df[1].apply(lambda x : temp_function(x))

In [21]:
df[1].unique()

array(['零售消费', 'IT/移动互联网', '电商', '金融', '其他', '游戏', '制造业', '汽车', 'IOT',
       '房地产', '医疗'], dtype=object)

In [22]:
collections.Counter(df[1])

Counter({'零售消费': 933,
         'IT/移动互联网': 2036,
         '电商': 644,
         '金融': 1143,
         '其他': 1382,
         '游戏': 163,
         '制造业': 164,
         '汽车': 98,
         'IOT': 92,
         '房地产': 159,
         '医疗': 137})

In [23]:
df.columns = ["aid","trade",*["c{}".format(v) for v in range(10)]]
df

,aid,trade,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
0,10113,零售消费,378020,378036,377965,377980,378047,378021,377903,377873,378033,378025
1,10140,IT/移动互联网,381349,381108,381299,381252,380842,381326,379187,381311,381059,380919
2,101966,电商,381295,381108,381284,381313,381106,380842,381316,381279,381315,381310
3,103927,金融,365636,365655,365723,365476,365546,365554,365423,365698,365331,365761
4,10714,IT/移动互联网,376192,376167,377536,376189,376126,374016,373949,378198,374017,379878
...,...,...,...,...,...,...,...,...,...,...,...,...
6946,8,零售消费,379195,379107,379194,380776,378982,379529,379133,379489,379526,379004
6947,83135,金融,374584,354849,377736,371117,377233,381026,379256,378084,379411,381309
6948,84249,IT/移动互联网,380858,379194,380797,380393,379607,380902,381078,379795,379977,378982
6949,89195,IT/移动互联网,358942,360290,360133,359550,368330,368145,363890,363980,367415,363881


In [25]:
article_dict = {}
with open(r"D:\zhaodachuan\data\predict_user_attribute20200911\raw_data\user_profile\title_tokenize.jsonl",encoding="utf8") as f:
    for line in f:
        line_dict = json.loads(line.strip())
        aid = line_dict["aid"]
        article_dict[aid] = {}
        article_dict[aid]["title_tokenize"] = line_dict["title_tokenize"]
with open(r"D:\zhaodachuan\data\predict_user_attribute20200911\raw_data\user_profile\content_tokenize.jsonl",encoding="utf8") as f:
    for line in f:
        line_dict = json.loads(line.strip())
        aid = line_dict["aid"]
        if not aid in article_dict:
            print(aid)
            continue
        article_dict[aid]["content_tokenize"] = line_dict["content_tokenize"][:512]

In [27]:
text_list = []
y_list = []
for index in df.index:
    y = df.loc[index,"trade"]
    temp_text_list = []
    for i in range(10):
        c_text_list = []
        c = "c{}".format(i)
        aid = str(df.loc[index,c]).strip()
        if aid:
            if aid in article_dict:
                c_text_list = [*article_dict[aid]["title_tokenize"],"<PADDING>",*article_dict[aid]["content_tokenize"][:512]]
            else:
                print("aid不存在",index,c,aid)
        else:
            print(index)
        temp_text_list.append(c_text_list)     
    text_list.append(temp_text_list)
    y_list.append(y)

aid不存在 55 c3 130
aid不存在 236 c8 161
aid不存在 290 c3 130
aid不存在 314 c8 142
aid不存在 912 c4 130
aid不存在 945 c6 134
aid不存在 1127 c1 108
aid不存在 2081 c2 130
aid不存在 2502 c5 130
aid不存在 3328 c8 126
aid不存在 3346 c3 130
aid不存在 3654 c5 130
aid不存在 3719 c2 213
aid不存在 5092 c1 130
aid不存在 6424 c3 130


In [28]:
with open(r"D:\zhaodachuan\data\predict_user_attribute20200911\raw_data\user_profile\trade\最终数据集tokenize.jsonl","w") as f:
    for tag,content_tokenize in zip(y_list,text_list):
        f.write(json.dumps({"tag":tag,"all_content_tokenize":content_tokenize},ensure_ascii=False) + "\n")